# Credibility Checker - A foray into detecting and separating fake news from real

### Overview: 
In this practical application, we answer the question 'What is the most effective model for determining whether a news article is real or fake?'

Following CRISP-DM method.

### Motivation:
NEWS has always been a critical source of truth and information of world events that affect everybody. In an era of widespread misinformation, distinguishing real news from fake is critical to maintaining an informed public. This project is an initial foray into classifying information vs disinformation by applying machine learning to classify news credibility, offering practical insights into automated fact-checking.

### The Data:

**Source** : The data is taken from Kaggle 'fake-and-real-news-dataset' (https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset/data).


The dataset contains 2 separate CSV files one for real and one for fake news articles. Both sheets contain 4 columns: Title, Text, Subject and Date.


Dataset columns:

* Title: title of news article
* Text: body text of news article
* Subject: subject of news article
* Date: publish date of news article



### Understanding the Data



According to the **Materials and Methods** section paper provided, the dataset is collected over **17** campaigns between Mat 2008 and November 2010.

### Read in the Data


In [70]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, PolynomialFeatures, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report, auc,PredictionErrorDisplay, mean_squared_error, f1_score, recall_score
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.inspection import permutation_importance

import warnings
warnings.simplefilter(action="ignore")

In [72]:
df = pd.read_csv("data/", sep = ';')

IsADirectoryError: [Errno 21] Is a directory: 'data/'

In [ ]:
df.head()

### Understanding the Features



In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

**Business Objective:**

### Engineering Features

### Train/Test Split

With your data prepared, split it into a train and test set.

### A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [31]:
# Naive Bayes Model

### A Simple Model

### Score the Model

What is the accuracy of your model?

### Model Comparisons


| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

### Problem 11: Improving the Model

GridSearchCV

#### Ensemble


#### Neural Network


#### Finding patterns If possible


#### Additional datasets If possible
